In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from scipy.interpolate import griddata

# ADCIRC variable names for water level and wave height
adcirc_waterlevel_name = 'zeta'
adcirc_waveheight_name = 'swan_HS'

# Base URL for the TDS data
TDSbase = "https://apsviz-tds.adcircprediction.org/thredds/dodsC/2024/coamps_09L/"
TDStime = "{}/ngom_rt_v19b/hpc6a.useast2/ngom_coamps_09L/nowcast/fort.63.nc"

# Time string
tstr = "2024092318"

# Form the specific URL
url = TDSbase + TDStime.format(tstr)

# Open the dataset
ds = xr.open_dataset(url)

# Extract grid data
lon_grid = ds['x'][:].values
lat_grid = ds['y'][:].values
elements = ds['element'][:].values - 1  # Convert to 0-based indexing
times = ds['time'][:].values

# Target location
lon_target = -76
lat_target = 32

# Locate the closest element
points = np.c_[lon_grid, lat_grid]
kdtree = KDTree(points)
_, elem_idx = kdtree.query([lon_target, lat_target])
element_nodes = elements[elem_idx]

# Extract time series at the element nodes
v1 = ds[adcirc_waterlevel_name][:, element_nodes[0]].values
v2 = ds[adcirc_waterlevel_name][:, element_nodes[1]].values
v3 = ds[adcirc_waterlevel_name][:, element_nodes[2]].values

# Perform linear interpolation
values = np.array([v1, v2, v3]).T
coords = np.c_[lon_grid[element_nodes], lat_grid[element_nodes]]
vi = griddata(coords, values, [[lon_target, lat_target]], method='linear').flatten()

# Plot the data
plt.figure()
plt.plot(times, v1, label='Node 1')
plt.plot(times, v2, label='Node 2')
plt.plot(times, v3, label='Node 3')
plt.plot(times, vi, label='Interpolated', linestyle='--')
plt.title(adcirc_waterlevel_name)
plt.ylabel('m')
plt.legend()
plt.show()

# To access wave heights, replace filename
url_wave = url.replace('fort', 'swan_HS')
ds_wave = xr.open_dataset(url_wave)

# Extract wave height time series
v1_wave = ds_wave[adcirc_waveheight_name][:, element_nodes[0]].values
v2_wave = ds_wave[adcirc_waveheight_name][:, element_nodes[1]].values
v3_wave = ds_wave[adcirc_waveheight_name][:, element_nodes[2]].values

vi_wave = griddata(coords, np.array([v1_wave, v2_wave, v3_wave]).T,
                   [[lon_target, lat_target]], method='linear').flatten()

# Plot the wave heights
plt.figure()
plt.plot(times, v1_wave, label='Node 1')
plt.plot(times, v2_wave, label='Node 2')
plt.plot(times, v3_wave, label='Node 3')
plt.plot(times, vi_wave, label='Interpolated', linestyle='--')
plt.title(adcirc_waveheight_name)
plt.ylabel('m')
plt.legend()
plt.show()

# Station File Example (fort.61.nc)
url_station = url.replace('63', '61')
ds_station = xr.open_dataset(url_station)

# Display station metadata
stations = ds_station['station_name'][:].values
print(stations)

# Find a specific station by name
target_station = '8775241'
station_idx = np.where(stations == target_station)[0][0]

# Extract data for the station
vi_station = ds_station[adcirc_waterlevel_name][:, station_idx].values

plt.figure()
plt.plot(ds_station['time'][:].values, vi_station)
plt.title(f"{adcirc_waterlevel_name} at station {target_station}")
plt.ylabel('m')
plt.show()


OSError: [Errno -70] NetCDF: DAP server error: 'https://apsviz-tds.adcircprediction.org/thredds/dodsC/2024/coamps_09L/2024092318/ngom_rt_v19b/hpc6a.useast2/ngom_coamps_09L/nowcast/fort.63.nc'